In [1]:
# setup the data reading. 
import pandas as pd
from pandas import read_parquet
#from google.colab import drive
#drive.mount('/content/drive')
import pandas as pd
import numpy as np
import math
from mip import Model, xsum, maximize, BINARY

In [2]:
data=pd.read_parquet("C:\\Mohammadhosein\\PhD Courses\\Fall Semester 2022\\RL and System Constrol\\For the project\\RL in ride sourcing\\Matching\\yellow_tripdata_2022-01.parquet")

In [3]:
#@title Default title text
class customers_data:
  def __init__(self,picTime,drpTime,passengers,tripDistance,\
               picLocation,drpLocation,fee,real_picTime,id):
    self.picTime = picTime
    self.drpTime = drpTime
    self.passengers = passengers
    self.tripDistance = tripDistance
    self.picLocation = picLocation
    self.drpLocation = drpLocation
    self.travel_time = drpTime - picTime
    self.fee = fee
    self.real_picTime = real_picTime
    self.id = id

class driver_data:
  def __init__(self,ID,status,location,remain_time):
    self.ID = ID
    self.status = status
    self.location = location
    self.remain_time = remain_time

#data = read_parquet('/content/drive/MyDrive/yellow_tripdata_2022-01.parquet')
#print(data)

In [4]:
# most frequent position 
#data = read_parquet('/content/drive/MyDrive/yellow_tripdata_2022-01.parquet')
position_stat = {}
for i in range(10000):
  cur_pos = data['PULocationID'][i]
  position_stat[cur_pos] = position_stat.get(cur_pos,0) + 1

Most_Frequent_20 = (sorted(position_stat.items(), key = lambda kv:(kv[1], kv[0]),reverse = True)) 
Most_Frequent_20 = Most_Frequent_20[:20]
print(Most_Frequent_20)
final_most_frequent_20 = []
for i in Most_Frequent_20:
  final_most_frequent_20.append(i[0])
print(final_most_frequent_20)

[(79, 621), (48, 471), (170, 424), (237, 369), (107, 348), (142, 337), (249, 326), (132, 321), (68, 319), (161, 318), (263, 313), (234, 306), (141, 297), (148, 288), (239, 276), (162, 252), (229, 247), (164, 240), (114, 239), (50, 222)]
[79, 48, 170, 237, 107, 142, 249, 132, 68, 161, 263, 234, 141, 148, 239, 162, 229, 164, 114, 50]


In [5]:
def initialize_drivers(driver_num,pos):
  drivers_map = {}
  drivers_stack_idle = []
  cnt = 0
  for j in pos:
    for _ in range(driver_num):
      cur_driver = driver_data(cnt,'idle',j,0)
      drivers_map[cnt] = cur_driver
      drivers_stack_idle.append(cur_driver)
      cnt += 1
  return drivers_map,drivers_stack_idle

drivers_map,drivers_stack_idle = initialize_drivers(5,final_most_frequent_20)
print(drivers_map)
#print(drivers_stack_idle)

{0: <__main__.driver_data object at 0x0000026311F956C8>, 1: <__main__.driver_data object at 0x0000026311E8BA88>, 2: <__main__.driver_data object at 0x0000026311F95808>, 3: <__main__.driver_data object at 0x0000026311F95888>, 4: <__main__.driver_data object at 0x0000026311F958C8>, 5: <__main__.driver_data object at 0x0000026311F95948>, 6: <__main__.driver_data object at 0x0000026311F95988>, 7: <__main__.driver_data object at 0x0000026311F959C8>, 8: <__main__.driver_data object at 0x0000026311F95A08>, 9: <__main__.driver_data object at 0x0000026311F95908>, 10: <__main__.driver_data object at 0x0000026311F95A48>, 11: <__main__.driver_data object at 0x0000026311F95A88>, 12: <__main__.driver_data object at 0x0000026311F95AC8>, 13: <__main__.driver_data object at 0x0000026311F95B08>, 14: <__main__.driver_data object at 0x0000026311F95B48>, 15: <__main__.driver_data object at 0x0000026311F95B88>, 16: <__main__.driver_data object at 0x0000026311F95BC8>, 17: <__main__.driver_data object at 0x00

In [6]:
#@title Default title text
import collections
#data = read_parquet('/content/drive/MyDrive/yellow_tripdata_2022-01.parquet')
day_map = collections.defaultdict(list)
#print(data)
dates = ['01']

for i in range(len(data)):
    cur_day = str(data['tpep_pickup_datetime'][i])[8:10]
    #print(data['DOLocationID'][i])
    #print(cur_day)
    if not cur_day in dates: continue
    real_picTime = str(data['tpep_pickup_datetime'][i])
    temp = str(data['tpep_pickup_datetime'][i]).split(' ')
    temp = temp[1].split(':')
    picHour, picMin = int(temp[0]), int(temp[1])
    picTime = picHour * 60 + picMin
    if picTime < 360 or picTime > 780: continue
    
    temp = str(data['tpep_dropoff_datetime'][i]).split(' ')
    temp = temp[1].split(':')
    drpHour, drpMin = int(temp[0]), int(temp[1])
    drpTime = drpHour * 60 + drpMin

    passengers = str(data['passenger_count'][i])
    tripDistance = str(data['trip_distance'][i])
    picLocation = str(data['PULocationID'][i])
    drpLocation = str(data['DOLocationID'][i])
    fee = float(data['total_amount'][i])
    cur_data = customers_data(picTime, drpTime, passengers,\
                              tripDistance, picLocation,drpLocation,fee,real_picTime,i)

    day_map[cur_day].append(cur_data)

In [7]:
print(len(day_map['01']))

10777


In [8]:
print(timeLine)
print(single_day_time_map[5][0].picLocation)
print(single_day_time_map[5][1].picLocation)

NameError: name 'timeLine' is not defined

In [8]:
import bisect
single_day_time_map = collections.defaultdict(list)
for key in day_map:
  for i in range(len(day_map[key])):
    cur_time = day_map[key][i].picTime
    cur_location = day_map[key][i].picLocation
    single_day_time_map[cur_time].append(day_map[key][i])

timeLine = sorted(single_day_time_map.keys())
new_time_map = {}
combine_time_line = []
for i in range(0,60 * 24, 10):
  new_time_map[i] = []
  combine_time_line.append(i)
#print(new_time_map)
for time in timeLine:
  indx = bisect.bisect_right(combine_time_line,time)
  for data in single_day_time_map[time]:
    new_time_map[combine_time_line[indx - 1]].append(data)

for key in new_time_map:
  print("%d:" %(key))
  for i in new_time_map[key]:
    print(i.picTime)
  print("=================")
#for i in range(len(single_day_time_map[25])):
#  print(single_day_time_map[25][i].drpLocation)

#print(timeLine)

0:
10:
20:
30:
40:
50:
60:
70:
80:
90:
100:
110:
120:
130:
140:
150:
160:
170:
180:
190:
200:
210:
220:
230:
240:
250:
260:
270:
280:
290:
300:
310:
320:
330:
340:
350:
360:
360
360
360
360
360
360
360
360
360
360
360
361
361
361
361
361
361
361
361
361
361
361
361
361
361
361
361
361
362
362
362
362
363
363
363
363
363
363
363
364
364
364
364
364
364
364
364
365
365
365
365
365
365
365
365
365
365
365
365
365
366
366
366
366
366
366
366
366
367
367
367
367
367
367
367
367
367
368
368
368
368
368
368
368
368
369
369
369
369
369
369
369
369
369
369
369
370:
370
370
370
370
370
370
370
370
370
370
370
371
371
371
371
371
372
372
372
372
372
372
372
372
373
373
373
373
373
373
373
373
373
373
373
373
373
374
374
374
374
374
374
374
374
374
374
374
374
375
375
375
375
375
375
375
375
375
375
375
375
375
375
376
376
376
376
376
376
376
376
376
376
377
377
377
377
377
377
377
377
378
378
378
378
378
378
378
378
378
378
378
378
378
378
378
378
379
379
379
379
379
379
379
379
379
379
379
379
3

In [9]:
#loading data
state_values=pd.read_csv('C:\Mohammadhosein\PhD Courses\Fall Semester 2022\RL and System Constrol\Exports\statevaluse.csv',header=None)
state_values=state_values.values
state_values=np.reshape(state_values,(265,42))

ODTT=pd.read_csv('C:\Mohammadhosein\PhD Courses\Fall Semester 2022\RL and System Constrol\Exports\ODTTime.csv',header=None)
ODTT=ODTT.values
ODTT=np.reshape(ODTT,(265,265))

# Action values

In [10]:
def action_value_function(state_value,ODTT,passn_list,driver_list,  gamma,currenttime):
    trip_requests=len(passn_list)
    available_driver=len(driver_list)
    action_value=np.zeros((available_driver,trip_requests))
    pass_driv_assg=np.zeros((available_driver,trip_requests))
    for i in range (0,available_driver):
        for j in range (0, trip_requests):
            action_value[i][j]= (gamma**(ODTT[driver_list[i][1]-1, passn_list[j][1]-1] +passn_list[j][4]))*state_value[ passn_list[j][2]-1 ][currenttime]-state_value[ driver_list[i][1]-1 ][currenttime]+passn_list[j][3]
            pass_driv_assg[i][j]=passn_list[j][0]
    action_value=action_value.T
    return action_value,pass_driv_assg

In [11]:
def fare_base_function(passn_list,driver_list):
    trip_requests=len(passn_list)
    available_driver=len(driver_list)
    action_value=np.zeros((available_driver,trip_requests))
    pass_driv_assg=np.zeros((available_driver,trip_requests))
    for i in range (0,available_driver):
        for j in range (0, trip_requests):
            action_value[i][j]= passn_list[j][3]
            pass_driv_assg[i][j]=passn_list[j][0]
    action_value=action_value.T
    return action_value,pass_driv_assg

# Matching

In [24]:
def matching(action_value,pass_driv_assg,trip_requests_list,available_driver_list):
    available_driver=len(available_driver_list)
    trip_requests=len(trip_requests_list)
    print (available_driver,trip_requests)
    model=Model()
    # trip * driver
    x=[[model.add_var(var_type=BINARY) for j in range (0,available_driver)] for i in range (0,trip_requests)]
    model.objective = maximize(xsum(action_value[i][j]*x[i][j] for i in range (0,trip_requests) for j  in range (0,available_driver)))
    # Each passenger receiev at most one matched driver
    for i in range (0,trip_requests):
        model += xsum(x[i][j] for j in range (0,available_driver) ) <= 1
    for j in range (0,available_driver):
        model += xsum(x[i][j] for i in range (0,trip_requests) ) <= 1
    solution=np.zeros((trip_requests,available_driver))
    model.optimize()
    for i in range (0,trip_requests):
        solution[i]= ([x[i][j].x for j in range (0,available_driver)])
    total_reward=np.sum(solution*action_value)
    pass_driv_assg=np.sum(solution*pass_driv_assg.T,axis=0)
    driver_assignment=pass_driv_assg=pass_driv_assg+(pass_driv_assg==0)*-1
    driver_assignment=np.reshape(driver_assignment,(len(driver_assignment),1 ) )
    drv_id=np.reshape(available_driver_list,( len(available_driver_list) ,2  ))
    drv_id=drv_id.T[0]
    drv_id=np.reshape(drv_id,(len(drv_id),1 ))
    driver_assignment=np.concatenate((drv_id,driver_assignment),axis=1)
    serverd_pass=np.sum(solution,axis=1)
    serverd_pass=np.reshape(serverd_pass, (len (serverd_pass),1))
    gained_revenue=np.dot(serverd_pass.T,np.reshape(trip_requests_list,(len(trip_requests_list),np.shape(trip_requests_list)[1] )).T[3])[0]
    n_servertrips=np.sum(serverd_pass)
    n_unserved=len(serverd_pass)-n_servertrips
    passn_id=np.reshape(trip_requests_list,(len(trip_requests_list),np.shape(trip_requests_list)[1] )).T[0]
    passn_id=np.reshape(passn_id,(len(passn_id),1 ))
    serverd_pass=np.concatenate((passn_id,serverd_pass),axis=1)
    n_vacant=len(driver_assignment)-n_servertrips
    # Outputs: matching of passengers, (2) matching of drivers, (3) number of serived tripes, (4) numbe of unserved trips, (5) number of vacant cars, (6) gained reward, (7) gained revenue
    return serverd_pass,driver_assignment, n_servertrips,n_unserved,n_vacant ,total_reward,gained_revenue

In [13]:
print(new_time_map.keys())

dict_keys([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000, 1010, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090, 1100, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 1280, 1290, 1300, 1310, 1320, 1330, 1340, 1350, 1360, 1370, 1380, 1390, 1400, 1410, 1420, 1430])


In [14]:
len(drivers_stack_idle)

100

In [54]:
drivers_map,drivers_stack_idle = initialize_drivers(30,final_most_frequent_20)
waiting_customers = []
available_drivers = []
serving_drivers = []
customer_dictionary = {}
driver_dictionary = {}
all_out_puts=[]
for driver in drivers_stack_idle:
  available_drivers.append([driver.ID,driver.location])
  driver_dictionary[driver.ID] = [driver.ID,driver.location]

#print(available_drivers)
for key in new_time_map:
    flag = 1
    '''
    print("waiting_customers",end = "")
    print(waiting_customers)
    print("available_drivers",end = "")
    print(available_drivers)
    print("serving_drivers",end = "")
    print(serving_drivers)
    print("customer_dictionary",end = "")
    print(customer_dictionary)
    print("driver_dictionary",end = "")
    print(driver_dictionary)
    print("==========================================")
    '''
    
    cur_round_time_stamp = int(int(key) / 10 - 36)
    print("TimeStamp:", end = "")
    print(cur_round_time_stamp)
    if cur_round_time_stamp < 0 or cur_round_time_stamp > 3: continue
    for customer in new_time_map[key]:
      waiting_customers.append([customer.id,int(customer.picLocation),int(customer.drpLocation),customer.fee,math.ceil(customer.travel_time/10) ])
      customer_dictionary[customer.id] = [customer.id,int(customer.picLocation),int(customer.drpLocation),customer.fee,math.ceil(customer.travel_time/10)  ]
    if len(waiting_customers)*len(available_drivers)!=0:
        action_value,pass_driv_assg=action_value_function(state_values,ODTT,waiting_customers,available_drivers,0.9,cur_round_time_stamp)
        #action_value,pass_driv_assg=fare_base_function(waiting_customers,available_drivers)
        match_data=matching(action_value,pass_driv_assg,waiting_customers,available_drivers)
        print ('Gained revenue: ', match_data[-1])
        print ("No. Costumers:", len(waiting_customers), "No. Drivers: ",len(available_drivers)  )
        all_out_puts=all_out_puts+[np.reshape(match_data[2:],(1,5))]
    else:
        flag = 0
        print ("No. Costumers:", len(waiting_customers), "No. Drivers: ",len(available_drivers)  )
        all_out_puts=all_out_puts+[match_data[2],len(waiting_customers) ,len(available_drivers) ,match_data[5] ,match_data[6]]
    
    

    for driver in serving_drivers:
        driver.remain_time -= 1
        if driver.remain_time <= 0:
            available_drivers.append([driver.ID,driver.location])
            serving_drivers.remove(driver)
    
    if not flag: continue
    for i in match_data[0]:
        if i[1]:
            index = customer_dictionary[i[0]]
            waiting_customers.remove(index)

    for i in match_data[1]:
        if i[1] != -1:
            index = driver_dictionary[i[0]]
            available_drivers.remove(index)
            driver_location = index[1]
            pic_location = customer_dictionary[i[1]][1]
            drp_location = customer_dictionary[i[1]][2]
            travel_time = customer_dictionary[i[1]][-1]
            remain_time = ODTT[driver_location-1, pic_location-1] + int(travel_time)
            cur_driver = driver_data(index[0],'serving',customer_dictionary[i[1]][2],remain_time)
            serving_drivers.append(cur_driver)
            driver_dictionary[index[0]] = [index[0],customer_dictionary[i[1]][2]]

all_out_puts=pd.DataFrame(np.reshape(all_out_puts,( len(all_out_puts)  ,5)),columns=['Served_Costumers','Waiting_Costumers','Idle_AVs','Gained_reward','Gained_revenue'])
all_out_puts.to_csv('C:\\Mohammadhosein\\PhD Courses\\Fall Semester 2022\\RL and System Constrol\\For the project\\Exports\\RL_30D')

TimeStamp:-36
TimeStamp:-35
TimeStamp:-34
TimeStamp:-33
TimeStamp:-32
TimeStamp:-31
TimeStamp:-30
TimeStamp:-29
TimeStamp:-28
TimeStamp:-27
TimeStamp:-26
TimeStamp:-25
TimeStamp:-24
TimeStamp:-23
TimeStamp:-22
TimeStamp:-21
TimeStamp:-20
TimeStamp:-19
TimeStamp:-18
TimeStamp:-17
TimeStamp:-16
TimeStamp:-15
TimeStamp:-14
TimeStamp:-13
TimeStamp:-12
TimeStamp:-11
TimeStamp:-10
TimeStamp:-9
TimeStamp:-8
TimeStamp:-7
TimeStamp:-6
TimeStamp:-5
TimeStamp:-4
TimeStamp:-3
TimeStamp:-2
TimeStamp:-1
TimeStamp:0
600 96
Gained revenue:  2985.1599999999994
No. Costumers: 96 No. Drivers:  600
TimeStamp:1
508 116
Gained revenue:  2740.0199999999995
No. Costumers: 116 No. Drivers:  508
TimeStamp:2
407 100
Gained revenue:  2277.22
No. Costumers: 100 No. Drivers:  407
TimeStamp:3
345 123
Gained revenue:  3135.8399999999997
No. Costumers: 123 No. Drivers:  345
TimeStamp:4
TimeStamp:5
TimeStamp:6
TimeStamp:7
TimeStamp:8
TimeStamp:9
TimeStamp:10
TimeStamp:11
TimeStamp:12
TimeStamp:13
TimeStamp:14
TimeStamp

In [71]:
driver_dictionary[1]

161